## Import libraries

In [12]:
import numpy as np
import torch
from scipy.special import softmax
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    confusion_matrix,
    mean_absolute_error,
)
from skorch import NeuralNetClassifier
from torch import cuda, nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torchvision import models
from torchvision.transforms.v2 import Compose, ToDtype, ToImage

from dlordinal.datasets import FGNet
from dlordinal.metrics import accuracy_off1, amae, mmae, ranked_probability_score
from dlordinal.output_layers.copoc import COPOC

## Import FGNet dataset

In [13]:
fgnet_train = FGNet(
    root="./datasets",
    download=True,
    train=True,
    transform=Compose([ToImage(), ToDtype(torch.float32, scale=True)]),
)

fgnet_test = FGNet(
    root="./datasets",
    download=True,
    train=False,
    transform=Compose([ToImage(), ToDtype(torch.float32, scale=True)]),
)

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified


## Model training
Fine-tuning of Resnet18 on the FGNet dataset using the Conformal Predictions for OC (COPOC) output layer(s) from

Dey, P., Merugu, S., & Kaveri, S. R. (2023). Conformal prediction sets for ordinal classification. Advances in Neural Information Processing Systems, 36, 879-899,

which enforce unimodality in the output probabilities in a non-parametric way.

In [14]:
device = "cuda" if cuda.is_available() else "cpu"

num_classes = len(fgnet_train.classes)

# Initialize ResNet18 model
model = models.resnet18(weights="IMAGENET1K_V1")

# Add COPOC layer
model.fc = nn.Sequential(nn.Linear(model.fc.in_features, num_classes), COPOC())
model = model.to(device)

# Skorch estimator
estimator = NeuralNetClassifier(
    module=model,
    criterion=CrossEntropyLoss().to(device),
    optimizer=Adam,
    lr=0.001,
    max_epochs=30,
    device=device,
    batch_size=200,
)

# Prepare training labels
y_train = torch.tensor(fgnet_train.targets, dtype=torch.long)

# Train model
estimator.fit(fgnet_train, y_train)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        1.9458       0.1056        2.9033  27.3399
      2        1.7937       0.1988        4.4783  25.9128
      3        1.7733       0.1056       14.8507  25.8848
      4        1.6589       0.0994        9.0877  25.1364
      5        1.4085       0.1429        3.2452  22.2829
      6        1.3573       0.1988        2.3565  25.1594
      7        1.2720       0.2174        2.0465  49.2505
      8        1.1944       0.2547        1.9980  51.6799
      9        1.0869       0.2919        1.4406  51.9515
     10        1.0338       0.3043        1.4702  20.6470
     11        0.9513       0.3168        1.3803  22.8812
     12        0.9263       0.2981        1.4277  21.5922
     13        0.8541       0.4037        1.3414  21.4255
     14        0.7964       0.4224        1.3526  21.9901
     15        0.7301       0.3851        1.3938  21.1809
     16       

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(6

## Evaluation
Focus of the evaluation is on checking the unimodality of the predictive probability distributions.

In [15]:
def is_unimodal(probs):
    """Check if a 1D array is unimodal (increases to a peak, then decreases)."""
    peak_idx = np.argmax(probs)
    # Increasing up to peak
    inc = np.all(np.diff(probs[: peak_idx + 1]) >= 0)
    # Decreasing after peak
    dec = np.all(np.diff(probs[peak_idx:]) <= 0)
    return inc and dec


def check_unimodality(y_pred):
    """Check unimodality for each row in y_pred and return the proportion."""
    unimodal_flags = np.array([is_unimodal(row) for row in y_pred])
    # Proportion of rows that are unimodal
    proportion = np.mean(unimodal_flags)
    print(
        f"Unimodal predictions: {np.sum(unimodal_flags)} / {len(y_pred)} ({proportion})"
    )
    return proportion


def calculate_metrics(y_true, y_pred):
    """Calculate various metrics given true labels and predicted probabilities."""
    if np.allclose(np.sum(y_pred, axis=1), 1):
        y_pred_proba = y_pred
    else:
        y_pred_proba = softmax(y_pred, axis=1)

    y_pred_max = np.argmax(y_pred, axis=1)

    # Metrics
    amae_metric = amae(y_true, y_pred)
    mmae_metric = mmae(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred_max)
    acc = accuracy_score(y_true, y_pred_max)
    acc_1off = accuracy_off1(y_true, y_pred)
    qwk = cohen_kappa_score(y_true, y_pred_max, weights="quadratic")
    rps = ranked_probability_score(y_true, y_pred_proba)
    # Check unimodality
    unimodal_prop = check_unimodality(y_pred_proba)

    metrics = {
        "ACC": acc,
        "1OFF": acc_1off,
        "MAE": mae,
        "QWK": qwk,
        "AMAE": amae_metric,
        "MMAE": mmae_metric,
        "RPS": rps,
        "Unimodality": unimodal_prop,
    }

    for key, value in metrics.items():
        print(f"{key}: {value}")

    print(confusion_matrix(y_true, y_pred_max))

    return metrics


# Evaluate on test set
test_probs = estimator.predict_proba(fgnet_test)
print(calculate_metrics(fgnet_test.targets, test_probs))

Unimodal predictions: 201 / 201 (1.0)
ACC: 0.46766169154228854
1OFF: 0.8805970149253731
MAE: 0.6567164179104478
QWK: 0.751150585385547
AMAE: 0.6882034632034632
MMAE: 1.0
RPS: 0.5108151327300118
Unimodality: 1.0
[[ 7 15  0  0  0  0]
 [ 0 27 20 12  1  0]
 [ 0  3 11 15  4  0]
 [ 0  0  3 31  6  2]
 [ 0  0  3  9 16  2]
 [ 0  0  0  2 10  2]]
{'ACC': 0.46766169154228854, '1OFF': 0.8805970149253731, 'MAE': 0.6567164179104478, 'QWK': 0.751150585385547, 'AMAE': 0.6882034632034632, 'MMAE': 1.0, 'RPS': 0.5108151327300118, 'Unimodality': 1.0}
